In [141]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('ggplot')

import pandas as pd
import numpy as np

from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score

import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [142]:
distance_df = pd.read_excel('distance_data_CP59.xlsx')
coordinate = pd.read_excel('distance_data_CP59.xlsx', sheet_name = 'coordinate')

In [143]:
distance_df_melt = pd.melt(distance_df, value_vars = ['CP21-O-0059'])

In [144]:
fig = px.histogram(distance_df_melt, x="value", color = 'variable')
fig.show()

# Coordinates Analysis

In [145]:
coordinate

,x,y,z
0,183.217896,87.184867,33.978806
1,185.318705,84.291655,34.752898
2,183.937586,85.290537,34.880400
3,184.985221,87.444619,32.622806
4,180.274859,94.066710,31.006901
...,...,...,...
787,212.526899,96.006812,20.152700
788,210.522049,96.898236,19.948826
789,213.361794,97.775398,19.448063
790,192.866370,99.365568,21.675800


In [146]:
px.scatter(coordinate ,x= coordinate['x'], y = coordinate['y'])

In [147]:
px.scatter_3d(coordinate ,x= coordinate['x'], y = coordinate['y'], z = coordinate['z'],
             title = 'CP21-O-0059')

## DBSCAN

 ### 3D Clustering

In [148]:
from sklearn.preprocessing import StandardScaler

In [149]:
XYZ = np.array(coordinate[['x', 'y', 'z']], dtype='float64')

In [150]:
model_3d = DBSCAN(eps= 2.75, min_samples= 4).fit(XYZ) #  eps = 2.7
class_predictions_3d = model_3d.labels_

coordinate['CLUSTERS_DBSCAN_3d'] = class_predictions_3d
coordinate['CLUSTERS_DBSCAN_3d'] = coordinate['CLUSTERS_DBSCAN_3d'].astype('object')

In [151]:
coordinate['CLUSTERS_DBSCAN_3d'] = np.where(coordinate['CLUSTERS_DBSCAN_3d'] != -1, 0, -1)
coordinate['CLUSTERS_DBSCAN_3d'] = coordinate['CLUSTERS_DBSCAN_3d'].astype('str')

In [152]:
print(f'Number of clusters found: {len(np.unique(class_predictions_3d))}')
print(f'Number of outliers found: {len(class_predictions_3d[class_predictions_3d==-1])}')

no_outliers = 0
no_outliers = np.array([(counter+2)*x if x==-1 else x for counter, x in enumerate(class_predictions_3d)])
print(f'Silhouette outliers as singletons: {silhouette_score(XYZ, no_outliers)}')

Number of clusters found: 16
Number of outliers found: 133
Silhouette outliers as singletons: -0.5729836228022981


In [153]:
px.scatter(coordinate ,
           x= coordinate['x'], y = coordinate['y'], color = coordinate['CLUSTERS_DBSCAN_3d'])

In [154]:
px.scatter_3d(coordinate ,x= coordinate['x'], y = coordinate['y'], z = coordinate['z'],
             color = 'CLUSTERS_DBSCAN_3d')

## Include unit vector and  angle of plan

In [155]:
import math

In [156]:
angle_df = pd.read_excel('distance_data_CP59_angle.xlsx', sheet_name = 'cartesian unit vector')
coor_df = pd.read_excel('distance_data_CP59_angle.xlsx', sheet_name = 'coordinate')

angle_df = angle_df[['x', 'y', 'z']]

In [157]:
angle_df_zneg = angle_df[angle_df['z'] < 0]
angle_df_zpos = angle_df[angle_df['z'] >= 0]

angle_df_zneg_trans = angle_df_zneg*-1
angle_df = angle_df_zpos.append(angle_df_zneg_trans)

In [158]:
angle_df['x_arccos'] = angle_df['x'].apply(math.acos)
angle_df['y_arccos'] = angle_df['y'].apply(math.acos)
angle_df['z_arccos'] = angle_df['z'].apply(math.acos)

angle_df['x_deg'] = angle_df['x_arccos'].apply(math.degrees)
angle_df['y_deg'] = angle_df['y_arccos'].apply(math.degrees)
angle_df['z_deg'] = angle_df['z_arccos'].apply(math.degrees)

In [159]:
angle_df_deg = angle_df[['x_deg', 'y_deg', 'z_deg']]
angle_df_deg

,x_deg,y_deg,z_deg
0,61.571350,57.738178,45.662865
1,62.783954,56.712806,45.595216
2,61.838140,57.075324,46.041990
3,63.031408,57.799477,44.405788
4,59.314253,61.803170,44.065599
...,...,...,...
696,77.013821,50.924567,42.005465
714,73.663267,57.390871,37.437587
757,79.047449,62.156895,30.279501
758,80.172875,62.137519,29.837500


In [160]:
angle_df_deg_long = pd.melt(angle_df_deg, value_vars = ['x_deg', 'y_deg', 'z_deg'])

In [161]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Histogram(x = angle_df_deg['x_deg'], marker_color = 'Orange'),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x = angle_df_deg['y_deg'], marker_color = 'Blue'),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x = angle_df_deg['z_deg'], marker_color = 'Green'),
    row=1, col=1
)
fig.add_trace(
    go.Box(y= angle_df_deg['x_deg'], name = 'X Deg',  marker_color = 'Orange'),
    row=1, col=2
)
fig.add_trace(
    go.Box(y= angle_df_deg['y_deg'], name = 'Y Deg', marker_color = 'Blue'),
    row=1, col=2
)
fig.add_trace(
    go.Box(y= angle_df_deg['z_deg'], name = 'Z Deg', marker_color = 'Green'),
    row=1, col=2
)

fig.update_layout(height=500, width=800, barmode = 'overlay', showlegend = False,
                  title_text="Distribution of Support Unit Vector Angle - Perpendicular")
fig.show()

## Calculate Upper Threshold of Z angle
- Upper Threshold = Q3
- Lower Threshold = Q1

In [162]:
def upperlimit_by_axis(df: pd.DataFrame, axis : str) -> float:
    axis_q3 = df[axis].quantile(0.75)
    axis_q1 = df[axis].quantile(0.25)
    axis_iqr = axis_q3 - axis_q1

    axis_upper = axis_q3 + 1.5*axis_iqr
    return axis_q3

def lowerlimit_by_axis(df: pd.DataFrame, axis : str) -> float:
    #axis_q3 = df[axis].quantile(0.75)
    axis_q1 = df[axis].quantile(0.25)
    #axis_iqr = axis_q3 - axis_q1

    #axis_upper = axis_q3 - 1.5*axis_iqr
    return axis_q1

In [163]:
z_deg_upper = upperlimit_by_axis(angle_df_deg, 'z_deg')
y_deg_upper = upperlimit_by_axis(angle_df_deg, 'y_deg')
x_deg_upper = upperlimit_by_axis(angle_df_deg, 'x_deg')

z_deg_lower = lowerlimit_by_axis(angle_df_deg, 'z_deg')

In [164]:
# Upper Side
#angle_df_deg['High_angle_flag'] = np.where(angle_df_deg['z_deg'] > z_deg_upper, -1, 0)

# Lower Side
angle_df_deg['High_angle_flag'] = np.where(angle_df_deg['z_deg'] <= z_deg_lower, -1, 0)

angle_df_deg['High_angle_flag'] = angle_df_deg['High_angle_flag'].astype('str')

In [165]:
coordinate_angle = pd.merge(coordinate, angle_df_deg, left_index = True, right_index = True)

In [166]:
coordinate_angle['DBSCAN_Angle'] = coordinate_angle['CLUSTERS_DBSCAN_3d'].astype('int') + \
                                   coordinate_angle['High_angle_flag'].astype('int')

coordinate_angle['DBSCAN_Angle_Flag'] = np.where(coordinate_angle['DBSCAN_Angle'] == -2, 1,
                                                np.where(coordinate_angle['DBSCAN_Angle'] == -1, 2, 0))

coordinate_angle['DBSCAN_Angle_Flag'] = coordinate_angle['DBSCAN_Angle_Flag'].astype('str')

In [167]:
px.scatter_3d(coordinate_angle ,x= coordinate_angle['x'], y = coordinate_angle['y'], z = coordinate_angle['z'],
             color = 'DBSCAN_Angle_Flag',
            color_discrete_map={
                "1": "red",
                "0": "blue",
                "2": "orange"})

In [168]:
px.scatter_3d(coordinate_angle ,x= coordinate_angle['x'], y = coordinate_angle['y'], z = coordinate_angle['z'],
             color = 'High_angle_flag',
            color_discrete_map={
                "-1": "red",
                "0": "blue"})

In [173]:
submit_df = coordinate_angle[['x', 'y', 'z', 'x_deg', 'y_deg','z_deg', 'CLUSTERS_DBSCAN_3d', 'High_angle_flag', 'DBSCAN_Angle_Flag']]

In [174]:
submit_df.to_excel('CP21-O-0059_density_highangle_result.xlsx', index = False)